Se aborda en esta primera parte del proyecto final *el problema de la determinación del redshift*.

Se muestra el mejor código pensado para resolver este problema y que en cuya arquitectura sean puestos en práctica los diferentes temas abordados a lo largo del curso.

Se dividirá el código en bloques con una tarea diferente, indicando su función y sus implicaciones.

Elaborado por: Oscar Raúl Sánchez Padilla

In [1]:
# BLOQUE 1: IMPORTACIONES Y CONFIGURACIÓN

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Scikit-learn: Servirá para separar los datos en train y test, hará la validación cruzada más delante, Normalizará y/o escalará los datos y calculará métricas.

#Usaremos la siguiente librería para la cross-validation
from sklearn.model_selection import train_test_split, KFold  

#StandardScaler estandarizará las características de tus datos. Evitará darles pesos diferentes a datos de diferentes tamaños o magnitudes.
#RobustScaler nos funcionara como alternativa a StandardScaler que maneja mejor outliers(valores atipicos o fuera de lo normal)
from sklearn.preprocessing import StandardScaler, RobustScaler

#Las siguientes funciones servirán para evaluar qué tan bueno será nuestro modelo de regresión comparando las predicciones con los valores reales. Son métricas de rendimiento.
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# TensorFlow y Keras
import tensorflow as tf

# Las componentes principales para construir y entrenar redes neuronales:
from tensorflow.keras import layers, models, callbacks, regularizers

# Usamos el siguiente algoritmo de optimización para entrenamiento de las redes neuronales:
from tensorflow.keras.optimizers import Adam

# Para reproducibilidad fijamos los numeros "aleatorios" para que siempre sean iguales.
# Esto es útil para saber si los cambios son por mi código y no por azar. 
np.random.seed(42)
tf.random.set_seed(42) #Usamos el número 42 como convención dentro del mundo de la programación, en general puede ser cualquier numero

# Esto hará que nuestras gráficas se vean más presentables (No afecta al código en si mismo)
plt.style.use('seaborn-v0_8-darkgrid')

In [2]:
# BLOQUE 2: CARGA Y PREPROCESAMIENTO DE DATOS

# Rutas de tus archivos (guardados localmente)
csv_path = r'C:\Users\op354\OneDrive\Documentos\Oscar\UG\Repositorio Github\ejerciosytareas-Oscarraul28\balanced_spectra_features.csv'
npz_path = r'C:\Users\op354\OneDrive\Documentos\Oscar\UG\Repositorio Github\ejerciosytareas-Oscarraul28\balanced_spectra_flux.npz'

# Carga de datos
print("📂 Cargando datos...")
df = pd.read_csv(csv_path) # Lee el archivo CSV y lo convierte en un DataFrame (tabla de pandas)
npz = np.load(npz_path, allow_pickle=True) # Carga el archivo NPZ (formato comprimido de NumPy). Es como un diccionario que contiene arrays.
targetids = npz['targetids'] # Extrae la lista de IDs de los objetos astronómicos
flujo_arr = npz['flux_arrays'] # Extrae los espectros (arrays de flujo luminoso), donde cada espectro es un array de muchisimos pixeles

print(f"   ✅ CSV cargado: {df.shape}") # Aquí van los datos pequeños
print(f"   ✅ NPZ cargado: {len(flujo_arr)} espectros") # Y aquí los datos grandes

# Construir dataset alineando CSV con NPZ. Esto es muy importante porque...
X_list = [] # Aquí se guardan los espectros (features)
y_list = [] # Aquí se guardan los redshifts!! Que son los targets!!
espectro_lista = [] #Aquí se guardan los tipos espectrales

#Aqui hacemos el loop por cada objeto, donde terminamos obteniendo tanto el indice como su respectivo valor
for i, tid in enumerate(targetids):
    row = df.loc[df['targetid'] == tid] #Aquí busca la fila en el CSV que tenga ese targetid
    if row.empty:
        continue #Si no encuentra el objeto en el CSV salta al siguiente
    redshift = row['redshift'].values[0] #Toma de la columna de redshift el primer (y unico) valor
    if pd.isna(redshift):
        continue #Aquí solo nos aseguramos de tener datos completos. Si el redshift es NaN, lo salta.
    X_list.append(flujo_arr[i])
    y_list.append(float(redshift))
    espectro_lista.append(row['spectype'].values[0])

# Guardamos todo:
X = np.asarray(X_list)         # Espectros 
y = np.asarray(y_list)         # Redshift (target)
spectypes = np.asarray(espectro_lista)  # Tipos espectrales (GALAXY, QSO, STAR,...)

print(f"\n📊 Dataset construido:")
print(f"   - Espectros: {X.shape}") #Muestra la cantidad de espectros con la cantidad pixeles que contienen todos por igual
print(f"   - Dimensión espectral: {X.shape[1]} píxeles") # shape[1] nuestra el segundo número = píxeles por espectro
print(f"   - Redshift min/max: [{y.min():.4f}, {y.max():.4f}]") # Rango de redshift en los datos mostrando 4 decimales
print(f"   - Tipos espectrales únicos: {len(np.unique(spectypes))}") # Encuentra valores únicos y nos dice cuantos tipos diferentes hay

# Normalización: dividir por máximo absoluto de cada espectro
eps = 1e-12 #Evita dividir por cero
X_norm = X / (np.max(np.abs(X), axis=1, keepdims=True) + eps) # Tomamos el valor absoluto de todos los numeros para normalizar por la intensidad máxima, con axis=1 = operamos en cada fila (cada espectro) y keepdims=True = mantiene la dimensión.
print(f"\n✅ Normalización por espectro completada")

# Escalado global con StandardScaler
scaler = StandardScaler() # Se crea el objeto escalador, fit calcula la media y desviación estándar de todos los datos y transform aplica la fórmula: (X - media) / desv_std
X_scaled = scaler.fit_transform(X_norm)
print(f"✅ StandardScaler aplicado")

# Convertir a formato 3D para CNN (samples, timesteps, features)
X_cnn = np.expand_dims(X_scaled, axis=-1) # Agrega una dimensión más para en la tercer dimensión incluir el canal que nos da el color. 1 canal grises, 3 canales color.
print(f"✅ Shape para CNN: {X_cnn.shape}")

# División train/test estratificada por tipo espectral
X_train, X_test, y_train, y_test = train_test_split(
    X_cnn, y, # Los datos a dividir: features (X) y target (y)
    test_size=0.2, # 20% para test, 80% para train
    random_state=42, # Semilla para reproducibilidad ya vista
    stratify=spectypes # Organiza los tipos espectrales para train y test en la misma proporción
)

print(f"\n🔀 División train/test:")
print(f"   - Train: {X_train.shape[0]} espectros ({100*len(X_train)/len(X):.1f}%)")
print(f"   - Test:  {X_test.shape[0]} espectros ({100*len(X_test)/len(X):.1f}%)")
print(f"\n✅ Datos listos para entrenar")

📂 Cargando datos...
   ✅ CSV cargado: (492, 10)
   ✅ NPZ cargado: 492 espectros

📊 Dataset construido:
   - Espectros: (492, 7781)
   - Dimensión espectral: 7781 píxeles
   - Redshift min/max: [-0.0011, 3.4567]
   - Tipos espectrales únicos: 3

✅ Normalización por espectro completada
✅ StandardScaler aplicado
✅ Shape para CNN: (492, 7781, 1)

🔀 División train/test:
   - Train: 393 espectros (79.9%)
   - Test:  99 espectros (20.1%)

✅ Datos listos para entrenar
